In [ ]:
#Standard imports for everything dataset related lol
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install tensorflow
# !pip install --use-pep517 pandas
# !pip install google-colab
# !pip install torch
# !pip install scikit-ensemble
# !pip install scikit-learn



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Actual NN stuff
import tensorflow as tf
import keras
# from google-colab import drive
from tensorflow import keras
from keras import layers
# drive.mount('/content/drive/')
import os
import torch
from torch import nn
from sklearn.ensemble import IsolationForest
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, LSTM






In [ ]:
FILENAME = "2023"; #Years range from 2016-2023
data = pd.read_csv(r"C:\Users\user\Downloads\dsc_fc_summed_spectra_2016_v01\dsc_fc_summed_spectra_2016_v01.csv",\
delimiter = ',', parse_dates=[0], \
infer_datetime_format=True, na_values='0', \
header = None)


##Data Importing

In [ ]:
# Global Variables to mess with stuff
DATA_YEARS = ['2016','2017'] #Make these sequential, but you can put any year here except 2023 because we didn't have Kp vals
SEQUENCE_SIZE = 1
BATCH_SIZE = 128
TRAIN_PERCENTAGE = 0.8

inputarr = []
targetarr = []
for DATA_YEAR in DATA_YEARS:
    #Read the data from a CSV into the notebook
    if(DATA_YEAR == '2018'):
        DATA_YEAR += '(cool)'

    inputcsv = pd.read_csv(
        r"C:\Users\user\Downloads\\"+DATA_YEAR+".csv",
        header=0,
        names=[str(i) for i in range(54)]
    )
    inputcsv.drop(columns='0', inplace=True)

    #Slap that data into a numpy array!
    inputarrpart = np.empty((inputcsv.shape[0],inputcsv.shape[1]))
    for i in range(inputcsv.shape[0]):
        inputarrpart[i] = inputcsv.iloc[i]
    inputarrpart[np.isnan(inputarrpart)] = 0

    #Getting real Kp values!! (2018 is still a little scuffed, don't worry about it)
    START_INDICES = {
        '2016': [0,262080],
        '2017': [527040],
        '2018(cool)': [1052640],
        '2019': [1578240],
        '2020': [2183040],
        '2021': [2630880],
        '2022': [3156480],
        '2023': ["DNE"],

    }
    END_INDICES = {
        '2016': [18720,527040],
        '2017': [1052640],
        '2018(cool)': [1578240],
        '2019': [1834560],
        '2020': [2630880],
        '2021': [3156480],
        '2022': [3682080],
        '2023': ["DNE"],

    }

    data = pd.read_csv(r"C:\Users\user\Downloads\kpapdata.txt", sep='\t')
    parsed_data = []
    # Split and parse the data
    with open(r"C:\Users\user\Downloads\kpapdata.txt", 'r') as file:
        lines = file.readlines()
    for line in lines:
        parts = line.split()
        date = ' '.join(parts[0:3])
        time = float(parts[3])
        kp = float(parts[7])  # Extract kp value (index 7 in the split parts)
        ap = float(parts[8])  # Extract ap value (index 8 in the split parts)
        parsed_data.append([date, time, kp, ap])

    # Create a DataFrame from the parsed data
    df = pd.DataFrame(parsed_data, columns=['Date', 'time', 'kp', 'ap'])
    # Print the resulting DataFrame
    filtered_df = df[df['Date'].str.startswith('2016 01 01')]

    # Convert 'Date' and 'time' columns to datetime format
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['time'].astype(str) + ':00')

    # Set 'Datetime' column as the index
    df.set_index('Datetime', inplace=True)

    # Resample and interpolate the data to every minute
    df_resampled = df.resample('T').interpolate()

    # Reset the index if you want to keep 'Datetime' as a column
    df_resampled.reset_index(inplace=True)


    kpList = df_resampled['kp'].tolist()
    totalList = []
    for n in range(len(START_INDICES[DATA_YEAR])):
        totalList += kpList[START_INDICES[DATA_YEAR][n]:END_INDICES[DATA_YEAR][n]]

    targetarrpart = np.zeros((inputcsv.shape[0]//SEQUENCE_SIZE,10))
    for n,target in enumerate(targetarrpart):
        target[round(totalList[n])] = 1

    if (not np.any(inputarr)):
        inputarr = inputarrpart
        targetarr = targetarrpart
    else:
        print("inputarr: "+str(np.shape(inputarr)))
        print(inputarr)
        print("inputarrpart: "+str(np.shape(inputarrpart)))
        print(inputarrpart)
        inputarr = np.concatenate((inputarr, inputarrpart), axis=0)
        targetarr = np.concatenate((targetarr, targetarrpart), axis=0)

#Here's what the numpy arrays looks like.
print("Example inputs:")
for i in range(5):
    print(inputarr[i])
print("# of input rows: "+str(inputarr.size // inputarr[0].size))

print("Example targets:")
for i in range(10):
    print(targetarr[i])
print("# of target rows: "+str(targetarr.size//targetarr[0].size))


inputarr: (283679, 53)
[[  6.76732  -3.30194 -12.9967  ...   0.        0.        0.     ]
 [  6.39107  -2.61173 -13.3271  ...   0.        0.        0.     ]
 [  6.44897  -2.61525 -13.3299  ...   0.        0.        0.     ]
 ...
 [ -5.78496  -2.02603  -2.64053 ...   0.        0.        0.     ]
 [ -3.60551  -5.31124  -3.16447 ...   0.        0.        0.     ]
 [ -4.61841  -4.5263   -3.05202 ...   0.        0.        0.     ]]
inputarrpart: (525599, 53)
[[-6.06788  -0.379552 -3.49708  ...  0.        0.        0.      ]
 [-5.8343   -3.03994  -2.98546  ...  0.        0.        0.      ]
 [-5.87318  -3.35291  -2.77476  ...  0.        0.        0.      ]
 ...
 [ 7.0421   -1.60274  -5.858    ...  0.        0.        0.      ]
 [ 6.74641   0.337871 -6.33095  ...  0.        0.        0.      ]
 [ 5.92092   1.64914  -7.022    ...  0.        0.        0.      ]]
Example inputs:
[  6.76732  -3.30194 -12.9967    0.        0.        0.        0.
   0.        0.        0.        0.        0.       

In [ ]:
#In case you want to know a few things about the data:
print("Inputs: "+str(inputarr.size // inputarr[0].size)+"\nValues per input: "+str(inputarr[0].size))
print("Outputs: "+str(targetarr.size // targetarr[0].size)+"\nValues per target: "+str(targetarr[0].size))

Inputs: 809278
Values per input: 53
Outputs: 809278
Values per target: 10


In [ ]:
#Setting up training and testing data arrays
trainInputArr = inputarr[0:int(len(inputarr)*TRAIN_PERCENTAGE)]
trainTargetArr = targetarr[0:int(len(inputarr)*TRAIN_PERCENTAGE)]
testInputArr = inputarr[int(len(inputarr)*TRAIN_PERCENTAGE):]
testTargetArr = targetarr[int(len(inputarr)*TRAIN_PERCENTAGE):]

## NN Stuff

Single-step dense


In [ ]:
#Where do you want to save the weights to?
WEIGHTS_FILEPATH = r"C:\Users\user\Desktop\single.h5"
print(targetarr)
#Creating a dataset to train on. This includes the raw data AND the targets per window!
train_dataset = tf.keras.utils.timeseries_dataset_from_array(
    trainInputArr,
    trainTargetArr,
    SEQUENCE_SIZE,
    batch_size = BATCH_SIZE
)
test_dataset = tf.keras.utils.timeseries_dataset_from_array(
    testInputArr,
    testTargetArr,
    SEQUENCE_SIZE,
    batch_size = BATCH_SIZE
)

'''
dataset params:
[
    [ [ [# of parameters] * SEQUENCE_SIZE ] * BATCH_SIZE ],
    [ [10 target K levels] * BATCH_SIZE ]
]
'''

# https://keras.io/api/layers/ has a wide variety of layers for us to use! Here's an example of how they work:
inputs = tf.keras.Input(shape=(SEQUENCE_SIZE,53,)) #This is the input layer. Note the lack of a previous input, like the others have.
x = tf.keras.layers.Flatten()(inputs)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x) #Uses 'x', the previous layer's output, as input. Node the softmax activation for the output (might change!).
model = tf.keras.Model(inputs=inputs, outputs=outputs) #This is the overall model. The layers show up here in 'outputs', which is based on previous layers.new_model = Sequential()
# new_model = Sequential()
# new_model.add(model)
# # new_model.add(Flatten())


# model = tf.keras.Sequential()
# model.add(tf.keras.Input(shape=(SEQUENCE_SIZE,53,)) #This is the input layer. Note the lack of a previous input, like the others have.)
# # model.add(tf.keras.layers.Flatten()(inputs))
# model.add(tf.keras.layers.Dense(64, activation='softmax')(x))
# model.add(tf.keras.layers.Dense(64, activation='softmax'))
# model.add(tf.keras.layers.Dense(10, activation='softmax'))



# val_performance['Dense'] = dense.evaluate(single_step_window.val)
# performance['Dense'] = dense.evaluate(single_step_window.test, verbose=0)





# https://keras.io/api/models/model_training_apis/ describes how to compile the model (like below!)
# Built in loss functions: https://keras.io/api/losses/ (BinaryCrossEntropy is a good bet prob)
# Build in optimizers: https://keras.io/api/optimizers/#:~:text=of%20available%20schedules.-,Available%20optimizers,-SGD
# Example compilation - we need to fill this in.
model.compile(loss= tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer= tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
print(model)

model.summary() #If this works, it should look great!


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 53)]           0         
                                                                 
 flatten (Flatten)           (None, 53)                0         
                                                                 
 dense (Dense)               (None, 10)                540       
                                                                 
Total params: 540 (2.11 KB)
Trainable params: 540 (2.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Actual training happens here, using the .fit function.
# Documentation: https://keras.io/api/models/model_training_apis/#:~:text=%5Bsource%5D-,fit%20method,-Model.fit
# x = input, y = targets (if x is a dataset, we don't need it since the targets are built into the dataset.)
# validation data is the test set (I think? It's what's used to evaluate loss, etc.)

TOTAL_EPOCHS = 1000
EPOCHS_PER_CYCLE = 1 #Should be a factor of TOTAL_EPOCHS
CYCLES = TOTAL_EPOCHS // EPOCHS_PER_CYCLE



for cycle in range(1,CYCLES+1):
    print("./-=oO0Oo=-\\.\nCycle",str(cycle),"/",str(CYCLES),"\n*\\-=oO0Oo=-/*")

    history = model.fit(

        x = train_dataset,
        epochs = EPOCHS_PER_CYCLE,
        validation_data = test_dataset,
        validation_steps = 30
    )

    model.save_weights(WEIGHTS_FILEPATH)
    print('Saved weights to '+WEIGHTS_FILEPATH)


# Loading weights is pretty easy:
# model.load_weights(wPath+"W_"+specs+'.h5')

multi step dense


In [ ]:
#Where do you want to save the weights to?
WEIGHTS_FILEPATH = r"C:\Users\user\Desktop\multi.h5"
print(targetarr)
#Creating a dataset to train on. This includes the raw data AND the targets per window!
train_dataset = tf.keras.utils.timeseries_dataset_from_array(
    trainInputArr,
    trainTargetArr,
    SEQUENCE_SIZE,
    batch_size = BATCH_SIZE
)
test_dataset = tf.keras.utils.timeseries_dataset_from_array(
    testInputArr,
    testTargetArr,
    SEQUENCE_SIZE,
    batch_size = BATCH_SIZE
)

'''
dataset params:
[
    [ [ [# of parameters] * SEQUENCE_SIZE ] * BATCH_SIZE ],
    [ [10 target K levels] * BATCH_SIZE ]
]
'''

# https://keras.io/api/layers/ has a wide variety of layers for us to use! Here's an example of how they work:
inputs = tf.keras.Input(shape=(SEQUENCE_SIZE,53,)) #This is the input layer. Note the lack of a previous input, like the others have.
x = tf.keras.layers.Flatten()(inputs)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x) #Uses 'x', the previous layer's output, as input. Node the softmax activation for the output (might change!).
model = tf.keras.Model(inputs=inputs, outputs=outputs) #This is the overall model. The layers show up here in 'outputs', which is based on previous layers.new_model = Sequential()
# new_model = Sequential()
# new_model.add(model)
# # new_model.add(Flatten())


# model = tf.keras.Sequential()
# model.add(tf.keras.Input(shape=(SEQUENCE_SIZE,53,)) #This is the input layer. Note the lack of a previous input, like the others have.)
# # model.add(tf.keras.layers.Flatten()(inputs))
# model.add(tf.keras.layers.Dense(64, activation='softmax')(x))
# model.add(tf.keras.layers.Dense(64, activation='softmax'))
# model.add(tf.keras.layers.Dense(10, activation='softmax'))


model = tf.keras.Sequential([
    tf.keras.Input(shape=(SEQUENCE_SIZE,53,)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='softmax'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='softmax'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

# val_performance['Dense'] = dense.evaluate(single_step_window.val)
# performance['Dense'] = dense.evaluate(single_step_window.test, verbose=0)





# https://keras.io/api/models/model_training_apis/ describes how to compile the model (like below!)
# Built in loss functions: https://keras.io/api/losses/ (BinaryCrossEntropy is a good bet prob)
# Build in optimizers: https://keras.io/api/optimizers/#:~:text=of%20available%20schedules.-,Available%20optimizers,-SGD
# Example compilation - we need to fill this in.
model.compile(loss= tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer= tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
print(model)

model.summary() #If this works, it should look great!


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_12 (Flatten)        (None, 53)                0         
                                                                 
 dense_12 (Dense)            (None, 64)                3456      
                                                                 
 flatten_13 (Flatten)        (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 flatten_14 (Flatten)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (

In [ ]:
# Actual training happens here, using the .fit function.
# Documentation: https://keras.io/api/models/model_training_apis/#:~:text=%5Bsource%5D-,fit%20method,-Model.fit
# x = input, y = targets (if x is a dataset, we don't need it since the targets are built into the dataset.)
# validation data is the test set (I think? It's what's used to evaluate loss, etc.)

TOTAL_EPOCHS = 1000
EPOCHS_PER_CYCLE = 1 #Should be a factor of TOTAL_EPOCHS
CYCLES = TOTAL_EPOCHS // EPOCHS_PER_CYCLE



for cycle in range(1,CYCLES+1):
    print("./-=oO0Oo=-\\.\nCycle",str(cycle),"/",str(CYCLES),"\n*\\-=oO0Oo=-/*")

    history = model.fit(

        x = train_dataset,
        epochs = EPOCHS_PER_CYCLE,
        validation_data = test_dataset,
        validation_steps = 30
    )

    model.save_weights(WEIGHTS_FILEPATH)
    print('Saved weights to '+WEIGHTS_FILEPATH)


# Loading weights is pretty easy:
# model.load_weights(wPath+"W_"+specs+'.h5')

In [ ]:
results = model.predict(
    test_dataset.take(1)
)

print(test_dataset)

print(results)

results.tofile(r"C:\user\user\Desktop\results", sep=',',format='%s')
print(type(results))
print("# of results: " + str(len(results)) + " | # of outputs per result: " + str(len(results[0])))
print("Sum of results (Expected 1 from BCE): "+ str(sum([i for i in results[0]])))

In [ ]:

# https://keras.io/api/layers/ has a wide variety of layers for us to use! Here's an example of how they work:
# inputs = tf.keras.Input(shape=(SEQUENCE_SIZE,53,)) #This is the input layer. Note the lack of a previous input, like the others have.
# x = tf.keras.layers.Flatten()(inputs)
# outputs = tf.keras.layers.Dense(10, activation='softmax')(x) #Uses 'x', the previous layer's output, as input. Node the softmax activation for the output (might change!).
# model = tf.keras.Model(inputs=inputs, outputs=outputs) #This is the overall model. The layers show up here in 'outputs', which is based on previous layers.new_model = Sequential()
# # new_model = Sequential()
# # new_model.add(model)
# # # new_model.add(Flatten())

input_shape = (SEQUENCE_SIZE, inputarr.shape[1])

model = tf.keras.Sequential()

model.add(LSTM(64, input_shape = input_shape))

model.add(Dense(10, activation = 'softmax'))

# https://keras.io/api/models/model_training_apis/ describes how to compile the model (like below!)
# Built in loss functions: https://keras.io/api/losses/ (BinaryCrossEntropy is a good bet prob)
# Build in optimizers: https://keras.io/api/optimizers/#:~:text=of%20available%20schedules.-,Available%20optimizers,-SGD
# Example compilation - we need to fill this in.
model.compile(loss= tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer= tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary() #If this works, it should look great!


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                30208     
                                                                 
 dense_15 (Dense)            (None, 10)                650       
                                                                 
Total params: 30858 (120.54 KB)
Trainable params: 30858 (120.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:

WEIGHTS_FILEPATH = r"C:\Users\User\Desktop\lstm1_weights.h5"



TOTAL_EPOCHS = 100
EPOCHS_PER_CYCLE = 1 #Should be a factor of TOTAL_EPOCHS
CYCLES = TOTAL_EPOCHS // EPOCHS_PER_CYCLE



for cycle in range(1,CYCLES+1):
    print("./-=oO0Oo=-\\.\nCycle",str(cycle),"/",str(CYCLES),"\n*\\-=oO0Oo=-/*")

    history = model.fit(

        x = train_dataset,
        epochs = EPOCHS_PER_CYCLE,
        validation_data = test_dataset,
        validation_steps = 30
    )

    model.save_weights(WEIGHTS_FILEPATH)
    print('Saved weights to '+WEIGHTS_FILEPATH)


# Loading weights is pretty easy:
# model.load_weights(wPath+"W_"+specs+'.h5')

In [ ]:
resultNum = 2
results = model.predict(
    train_dataset.take(resultNum)
)
targetks = []
outputks = []
for n,i in enumerate(results):
  targetks += [np.argmax(targetarr[n])]
  outputks += [np.argmax(i)]
print(targetks)
print(outputks)
t = np.array([i+1 for i in range(resultNum*128)])
plt.plot(t,outputks, color='blue')
plt.plot(t,targetks, color='green')
plt.show